# Outline
1. [About The Data](#aboutdata)
2. [Prepare The Preparation](#preparation)
3. [Look Into Data](#data)
4. [Preprocessing](#preprocessing)
5. [Building Model](#buildmodel)
6. [Fitting Model](#fitmodel)
7. [The Insight](#insight)

<a id='aboutdata'></a>
## 1. About The Data

Dataset yang akan digunakan berisi tentang informasi klien dari distributor barang dagang. Kolomnya berisi pendapatan dari setiap kategori produk dengan satuan moneter (seperti rupiah). Kolom yang tersedia diantaranya:
* FRESH : pendapatan tahunan dari produk kategori 'fresh' (Kontinu)
* MILK : pendapatan tahunan dari produk kategori 'milk' (Kontinu)
* GROCERY : pendapatan tahunan dari produk kategori 'grocery' (Kontinu)
* FROZEN : pendapatan tahunan dari produk kategori 'detergents paper' (Kontinu)
* DETERGENTS_PAPER : pendapatan tahunan dari produk kategori 'delicatessen' (Kontinu)
* DELICATESSEN : pendapatan tahunan dari produk kategori 'channel' (Kontinu)
* CHANNEL : golongan pembeli 1-HoReCa (Hotel/Restaurant/Cafe) atau 2-Retail (Nominal)
* REGION : regional pembeli 1-Lisnon, 2-Oporto atau 3-Other (Nominal)

Untuk informasi lebih tentang datanya dapat diakses di [UCI Wholesale customers Data Set](https://archive.ics.uci.edu/ml/datasets/wholesale+customers)

<a id='preparation'></a>
## 2. Preparation

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("/kaggle/input/wholesale-customers-data-set/Wholesale customers data.csv")

<a id='data'></a>
## 3. Data

In [ ]:
display(data.sample(4))
data.info()

<a id='preprocessing'></a>
## 4. Preprocessing

In [ ]:
data["Channel"]=data["Channel"].astype('category')
data["Region"] = data["Region"].astype('category')
data.rename(columns={'Delicassen':'Delicatessen'}, inplace=True)

In [ ]:
data.head()

In [ ]:
data_kontinu = data.select_dtypes('int64')
data_kontinu.head()

In [ ]:
data_kontinu.describe()

<a id='buildmodel'></a>
## 5. Build Model

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
for i,kol in enumerate(data_kontinu.columns):
    plt.subplot(2,3,i+1)
    plt.title(kol)
    plt.hist(data_kontinu[kol])
    plt.yticks(())
plt.tight_layout()

In [ ]:
from sklearn.cluster import KMeans

sse = []
list_k = list(range(1, 20))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(data_kontinu)
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(8, 8))
plt.plot(list_k, sse, '-o')
plt.xticks(list_k, list(range(1,20,1)))
plt.xlabel('K')
plt.ylabel('sse');

<a id='fitmodel'></a>
## 6. Fit Model

In [ ]:
k=5
model = KMeans(n_clusters=k, random_state=27)

model.fit(data_kontinu)

prediksi = model.predict(data_kontinu)

hasil = data_kontinu.copy()
hasil['klaster'] = prediksi
hasil.head()

In [ ]:
pusat = pd.DataFrame(data=model.cluster_centers_, columns=data_kontinu.columns)
pusat.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(data_kontinu.values)

normalized_data = scaler.transform(data_kontinu.values)
normalized_pusat = scaler.transform(pusat)

df_normalized = pd.DataFrame(data=normalized_data, columns=data_kontinu.columns)
df_normalized['klaster'] = prediksi.reshape(440,1)

df_normalized.head()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2).fit(normalized_data)

data_pca = pca.transform(normalized_data)
pusat_pca = pca.transform(normalized_pusat)

df_pca = pd.DataFrame(data=data_pca, columns = ['pc1', 'pc2'])
df_pca['klaster'] = prediksi.reshape(len(prediksi),1)

df_pca.head()

In [ ]:
plt.figure(figsize=(8,8))
markers = ["s" , "d" , "X" , "o" , "+"]
for i in range(k):
    plot_data = df_pca[df_pca['klaster']==i]
    plt.scatter(plot_data['pc1'],plot_data['pc2'],marker=markers[i],label=i)
    plt.scatter(pusat_pca[i][0],pusat_pca[i][1],marker=markers[i],s=100,c='black')
    
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

<a id='insight'></a>
## 7. The Insight

In [ ]:
bar = pusat.T.copy()
bar['lower'] = data_kontinu.quantile(0.25)
bar['mid']   = data_kontinu.quantile(0.5)
bar['upper'] = data_kontinu.quantile(0.75)

fig, axes = plt.subplots(nrows=1, ncols=6)
for i,b in enumerate(bar.index):
    bar.loc[[b]].plot(kind='bar',figsize=(25,4),sort_columns=True,ax=axes[i],xticks=(()),yticks=(()),title=b)

- Klaster 0 : Klaster dengan seluruh penjualan produknya sesuai rata-rata
- Klaster 1 : Klaster dengan penjualan cukup tinggi produk Fresh, Frozen, Delicatessen, sedangkan penjualan produk Milk, Grocery, Detegents_Paper biasa saja
- Klaster 2 : Klaster dengan penjualan produk Fresh. Frozen, Delicatessen sangat tinggi (mencakup outlier/pencilan), penjualan produk Milk cukup tinggi, dan penjualan produk Grocery dan Detergents_Paper biasa saja
- Klaster 3 : Klaster dengan penjualan produk Milk, Grocery, Detergents_Paper sangat tinggi, walaupun penjualan produk Fresh, Frozen, Delicatessen cukup tinggi
- Klaster 4 : Klaster dengan penjualan cukup tinggi produk Milk, Grocery, Detergents_Paper, Delicatessen, sedangkan penjualan produk Fresh, Frozen biasa saja